<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive_Bayes Basic Example_뉴스 데이터 분류

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [8]:
news = fetch_20newsgroups(subset = 'all')
X = news.data
y = news.target

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('vect', TfidfVectorizer(stop_words="english")),
    ('model', MultinomialNB()),
])

In [10]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
# 각 fold별 정확도 출력
print(scores)

[0.87984085 0.89493234 0.88485009 0.88007429 0.87689042]


## Naive_Bayes_From Scratch
참조 : 밑바닥부터 시작하는 데이터 과학 2판


나이브 베이즈의 핵심은, 각 독립변수들이 모두 독립이라는 커다란 가정 하에에에서 출발한다.  
어떤 원리로 모델이 만들어졌는지만 간단히 파악하여본다.

In [15]:
# 메세지를 단어 단위로 잘라주기
def tokenize(text):
  text = text.lower()
  all_words = re.findall("[a-z0-9']+", text)
  return set(all_words)

In [ ]:
class NaiveBayesClassifier:
  def __init__(self, k):
    self.k = k
    self.tokens = set()
    self.token_spam_counts = defaultdict(int)
    self.token_ham_counts = defalutdict(int)
    self.spam_messages = 0
    self.ham_messages = 0

  def train(self, messages):
    for message in messages:
      if message.is_spam:
        self.spam_messages += 1
      else:
        self.ham_messages += 1
      
      # 단어별로 메세지 종류에 따라 spam or ham 증가
      for token in tokenize(message.text):
        self.tokens.add(token)
        if message.is_spam:
          self.token_spam_counts[token] += 1
        else:
          self.token_ham_counts[token] += 1

  def prob(self, token):
    spam = self.token_spam_counts[token]
    ham = self.token_spam_counts[token]
    prob_spam = (spam + self.k) / (self.spam_messages + 2*self.k)
    prob_ham = (ham + self.k) / (self.ham_messages + 2*self.k)

    return prob_spam, prob_ham

  def predict(self, text):
    text_tokens = tokenize(text)
    log_prob_if_spam = 0
    log_prob_if_ham = 0

    for token in self.tokens:
      prob_spam, prob_ham = self.prob(token)

      # 메시지가 있으면 로그 확률 값을 더한다.
      if token in toext_tokens:
        log_prob_if_spam += math.log(prob_spam)
        log_prob_if_ham += math.log(prob_ham)
      # 아니라면 등장하지 않을 확률을 더한다.
      else:
        log_prob_if_spam += math.log(1-prob_spam)
        log_prob_if_ham += math.log(1-prob_ham)     

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_ham = math.exp(log_prob_if_ham)

    return prob_if_spam / (prob_if_spam + prob_if_ham)

여기서, self.k 는 smoothing에 해당된다.
가짜 빈도수를 추가하여, 특정 단어가 하나의 클래스에만 있는 경우에 발생하는 문제를 해결한다.